#Trabajo Práctico N° 1 - Parte 3 - Comunicación y Sincronismo

Integrantes:

- Emanuel Rodriguez
- Ezequiel Catania
- Julian Castellana
- Miguel Ledesma
- Santiago Galo

# Ejercicio 1


Para resolver este problema, se usó un enfoque basado en procesos independientes (uno por cada camión) y semáforos para manejar el acceso a recursos compartidos.

Cada recurso físico (plantas, zonas de carga/descarga, estación de servicio) está modelado con semáforos, que limitan cuántos camiones pueden usarlos al mismo tiempo. Por ejemplo:

-Las plantas permiten hasta 2 camiones simultáneamente.

-Las zonas de carga y descarga permiten solo 1 camión a la vez.

-Se usó una estación de servicio compartida con capacidad limitada.

Además, para controlar la cantidad total de viajes, se usó una variable compartida protegida con un lock (mutex_viajes). Lo mismo para acumular el tiempo total estimado (mutex_tiempo), evitando condiciones de carrera.

In [ ]:
%%writefile Truck.py

from multiprocessing import Process, Semaphore, Value, Lock
import time
import argparse
import random

LOADING_TIME = 2
UNLOADING_TIME = 2
REFUEL_TIME = 1


def truck(
    truck_id,
    flour_zone_tapiales,
    coal_zone_tapiales,
    flour_zone_fernandez,
    coal_zone_fernandez,
    tapiales_plant,
    fernandez_plant,
    gas_station,
    time_lock,
    trip_lock,
    remaining_trips,
    total_time,
    total_trips,
):
    while True:

        with trip_lock:
            if remaining_trips.value <= 0:
                break
            remaining_trips.value -= 1
            trip_number = total_trips - remaining_trips.value
            print(f"Camión {truck_id}: realizando viaje #{trip_number}")


        tapiales_plant.acquire()
        flour_zone_tapiales.acquire()
        print(f"Camión {truck_id}: cargando harina en Tapiales...")
        time.sleep(LOADING_TIME)
        flour_zone_tapiales.release()
        tapiales_plant.release()


        travel_outbound = random.randint(18, 24)
        print(f"Camión {truck_id}: viajando a Fernández por {travel_outbound} hs")
        time.sleep(travel_outbound / 10)


        fernandez_plant.acquire()
        coal_zone_fernandez.acquire()
        print(f"Camión {truck_id}: descargando harina en Fernández...")
        time.sleep(UNLOADING_TIME)
        coal_zone_fernandez.release()

        flour_zone_fernandez.acquire()
        print(f"Camión {truck_id}: cargando carbón en Fernández...")
        time.sleep(LOADING_TIME)
        flour_zone_fernandez.release()

        gas_station.acquire()
        print(f"Camión {truck_id}: cargando combustible...")
        time.sleep(REFUEL_TIME)
        gas_station.release()

        fernandez_plant.release()


        travel_return = random.randint(18, 24)
        print(f"Camión {truck_id}: viajando de vuelta a Tapiales por {travel_return} hs")
        time.sleep(travel_return / 10)


        tapiales_plant.acquire()
        coal_zone_tapiales.acquire()
        print(f"Camión {truck_id}: descargando carbón en Tapiales...")
        time.sleep(UNLOADING_TIME)
        coal_zone_tapiales.release()
        tapiales_plant.release()


        with time_lock:
            total_time.value += (
                travel_outbound
                + travel_return
                + LOADING_TIME * 2
                + UNLOADING_TIME * 2
                + REFUEL_TIME
            )

    print(f"Camión {truck_id}: terminó sus viajes.")


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Simulación de camiones de carga")
    parser.add_argument("trucks", type=int, help="Cantidad de camiones")
    parser.add_argument("trips", type=int, help="Cantidad total de viajes")
    args = parser.parse_args()

    num_trucks = args.trucks
    total_trips = args.trips


    flour_zone_tapiales = Semaphore(1)
    coal_zone_tapiales = Semaphore(1)
    flour_zone_fernandez = Semaphore(1)
    coal_zone_fernandez = Semaphore(1)

    tapiales_plant = Semaphore(2)
    fernandez_plant = Semaphore(2)
    gas_station = Semaphore(2)


    total_time = Value("i", 0)
    remaining_trips = Value("i", total_trips)
    time_lock = Lock()
    trip_lock = Lock()


    trucks = [
        Process(
            target=truck,
            args=(
                i,
                flour_zone_tapiales,
                coal_zone_tapiales,
                flour_zone_fernandez,
                coal_zone_fernandez,
                tapiales_plant,
                fernandez_plant,
                gas_station,
                time_lock,
                trip_lock,
                remaining_trips,
                total_time,
                total_trips,
            ),
        )
        for i in range(num_trucks)
    ]

    for t in trucks:
        t.start()
    for t in trucks:
        t.join()

    print("\n--- SIMULACIÓN COMPLETA ---")
    print(f"Tiempo total estimado (en horas): {total_time.value}")
    print(f"Tiempo total estimado (en días): {total_time.value / 24:.2f}")



Overwriting Truck.py


In [ ]:
!python Truck.py 5 10

Camión 0: realizando viaje #1
Camión 0: cargando harina en Tapiales...
Camión 1: realizando viaje #2
Camión 3: realizando viaje #3
Camión 2: realizando viaje #4
Camión 4: realizando viaje #5
Camión 1: cargando harina en Tapiales...
Camión 0: viajando a Fernández por 20 hs
Camión 3: cargando harina en Tapiales...
Camión 1: viajando a Fernández por 23 hs
Camión 0: descargando harina en Fernández...
Camión 0: cargando carbón en Fernández...
Camión 3: viajando a Fernández por 21 hs
Camión 2: cargando harina en Tapiales...
Camión 1: descargando harina en Fernández...
Camión 0: cargando combustible...
Camión 2: viajando a Fernández por 21 hs
Camión 4: cargando harina en Tapiales...
Camión 1: cargando carbón en Fernández...
Camión 0: viajando de vuelta a Tapiales por 19 hs
Camión 3: descargando harina en Fernández...
Camión 4: viajando a Fernández por 22 hs
Camión 1: cargando combustible...
Camión 0: descargando carbón en Tapiales...
Camión 3: cargando carbón en Fernández...
Camión 1: viajand

# Ejercicio 2

In [ ]:
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,934 kB]
Get:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,363 kB]
Get:13 http://archive.ubuntu.com/ubuntu ja

In [ ]:
!apt-get install g++

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:11.2.0-1ubuntu1).
g++ set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 89 not upgraded.


In [ ]:
%%writefile ejercicio2.cpp
#include <iostream>
#include <thread>
#include <mutex>
#include <vector>
#include <chrono>
#include <random>

using namespace std;

#define MAX_DOUGH_ON_TABLE 20
#define MAX_BREAD_IN_BASKET 50
#define MAX_PACKAGES_ON_COUNTER 30
#define INITIAL_LABELERS_AVAILABLE 2
#define MAESTRO_PRODUCES 2
#define ASSISTANT_PRODUCES 1
#define BAKER_CONSUMES 5
#define PACKER_CONSUMES 3
#define MIN_PACKAGES_TO_BUY 1
#define MAX_PACKAGES_TO_BUY 3
#define NUMBER_OF_CLIENTS 10

int doughOnTable = 0;
mutex doughMutex;

int breadInBasket = 0;
mutex basketMutex;

int packagesOnCounter = 0;
mutex counterMutex;

int labelersAvailable = INITIAL_LABELERS_AVAILABLE;
mutex labelerMutex;

int packagesSold = 0;
mutex salesMutex;

random_device rd;
mt19937 gen(rd());
uniform_int_distribution<> distClients(MIN_PACKAGES_TO_BUY, MAX_PACKAGES_TO_BUY);

void masterBaker() {
    while (true) {
        doughMutex.lock();
        if (doughOnTable <= MAX_DOUGH_ON_TABLE - MAESTRO_PRODUCES) {
            doughOnTable += MAESTRO_PRODUCES;
        }
        doughMutex.unlock();
    }
}

void assistantBaker() {
    while (true) {
        doughMutex.lock();
        if (doughOnTable < MAX_DOUGH_ON_TABLE) {
            doughOnTable += ASSISTANT_PRODUCES;
        }
        doughMutex.unlock();
    }
}

void baker() {
    while (true) {
        doughMutex.lock();
        basketMutex.lock();
        if (doughOnTable >= BAKER_CONSUMES && breadInBasket <= MAX_BREAD_IN_BASKET - BAKER_CONSUMES) {
            doughOnTable -= BAKER_CONSUMES;
            breadInBasket += BAKER_CONSUMES;
        }
        basketMutex.unlock();
        doughMutex.unlock();
    }
}

void packer() {
    while (true) {
        basketMutex.lock();
        counterMutex.lock();

        if (breadInBasket >= PACKER_CONSUMES && packagesOnCounter < MAX_PACKAGES_ON_COUNTER) {
            breadInBasket -= PACKER_CONSUMES;
            packagesOnCounter++;

            counterMutex.unlock();
            basketMutex.unlock();

            labelerMutex.lock();

            if (labelersAvailable > 0) {
                labelersAvailable--;
                labelerMutex.unlock();

                labelerMutex.lock();
                labelersAvailable++;
                labelerMutex.unlock();
            } else {
                labelerMutex.unlock();
            }
        } else {
            counterMutex.unlock();
            basketMutex.unlock();
        }
    }
}

void client() {
    int packagesToBuy = distClients(gen);
    int packagesBought = 0;
    while (packagesBought < packagesToBuy) {
        counterMutex.lock();
        if (packagesOnCounter > 0) {
            packagesOnCounter--;
            counterMutex.unlock();

            packagesBought++;

            salesMutex.lock();
            packagesSold++;
            salesMutex.unlock();
        } else {
            counterMutex.unlock();
        }
    }
}

int main() {
    thread t1(masterBaker);
    thread t2(assistantBaker);
    thread t3(baker);
    thread t4(packer);
    thread t5(packer);

    vector<thread> clients;
    for (int i = 0; i < NUMBER_OF_CLIENTS; i++) {
        clients.emplace_back(client);
    }

    for (auto& c : clients) {
        c.join();
    }

    cout << "\nTOTAL PAQUETES VENDIDOS: ";
    cout << packagesSold << endl;

    return EXIT_SUCCESS;;
}


Overwriting ejercicio2.cpp


In [ ]:
!g++ ejercicio2.cpp -o ejercicio2

In [ ]:
!./ejercicio2


TOTAL PAQUETES VENDIDOS: 23
terminate called without an active exception


# Ejercicio 3

Primero descargamos las dependencias necesarias para compilar Java

In [ ]:
!apt update
!apt install default-jdk -y

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,387 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,245 kB]
Get:13 http://security.ubuntu.com/ubun

Luego creamos el directorio donde se ejecutará el programa

In [ ]:
!mkdir -p Ejercicio3

In [ ]:
%%writefile Ejercicio3/Main.java

// File: Ejercicio3/Main.java
package Ejercicio3;

import java.util.concurrent.*;

public class Main
{
	private static final int TIME = 100;
	private static final int NUM_ASSISTANTS = 2;

	private static final int PARKING_SIZE = 6;
	private static final int PITS_SIZE = 3;
	private static final int SERVICE_SIZE = 2;


	private static Semaphore parkingMTX = new Semaphore(PARKING_SIZE);
	private static Semaphore pitsMTX = new Semaphore(PITS_SIZE);
	private static Semaphore serviceMTX = new Semaphore(SERVICE_SIZE);


	private static BlockingQueue<String> parkedCars = new LinkedBlockingQueue<>();
	private static BlockingQueue<String> checkedCars = new LinkedBlockingQueue<>();
	private static BlockingQueue<String> pitsCars = new LinkedBlockingQueue<>();
	private static BlockingQueue<String> fixedCars = new LinkedBlockingQueue<>();

	private static int cars = 3;

	public static void main(String[] args)
	{

		Richard richard = new Richard();
		Aaron aaron = new Aaron();
		Assistant assistant1 = new Assistant("Rob");
		Assistant assistant2 = new Assistant("Bran");
		Charles charles = new Charles();

		richard.start();
		aaron.start();
		assistant1.start();
		assistant2.start();
		charles.start();

	}

	static class Richard extends Thread
	{

		public void run()
		{
			try
			{
				while(thereCars())
				{

					parkingMTX.acquire();
					goParking();
					parkingMTX.release();
				}

				System.out.println("Richard avisa que no hay mas autos en la calle.");

				parkedCars.put("FIN");
			}
			catch (InterruptedException e)
			{
				System.out.println("ERROR: Richard no pudo realizar su trabajo.");
			}

		}

		private boolean thereCars()
		{
			return cars > 0;
		}

		private void goParking() throws InterruptedException
		{
			System.out.println("Richard lleva un auto al estacionamiento.");

			Thread.sleep(TIME);

			parkedCars.put("AUTO");

			cars --;
		}

	}

	static class Aaron extends Thread
	{

		public void run()
		{
			String nextCar;

			try
			{
				nextCar = parkedCars.take();

				while(!nextCar.equals("FIN"))
				{
					checkCar();

					nextCar = parkedCars.take();
				}

				System.out.println("Aaron avisa que no hay mas autos en el estacionamiento.");

				for (int i = 0; i < NUM_ASSISTANTS; i ++)
				{
				    checkedCars.put("FIN");
				}
			}
			catch (InterruptedException e)
			{
				System.out.println("ERROR: Aaron no pudo realizar su trabajo.");
			}
		}

		private void checkCar() throws InterruptedException
		{
			System.out.println("Aaron comienza la inspección del auto.");

			Thread.sleep(TIME);

			checkedCars.put("AUTO");
		}

	}

	static class Assistant extends Thread
	{
		String name = "";

		public Assistant(String name)
		{
			this.name = name;
		}

		public void run()
		{
			try
			{
				goPitBase();

				goServiceBase();
			}
			catch (InterruptedException e)
			{
				System.out.println("ERROR: El asistente " + this.name + " no pudo realizar su trabajo.");
			}
		}

		private void goPitBase() throws InterruptedException
		{
			String nextCar;

			nextCar = checkedCars.take();

			while(!nextCar.equals("FIN"))
			{
				pitsMTX.acquire();
				goPit();

				nextCar = checkedCars.take();
			}

			System.out.println("El asistente " + this.name + " avisa que no hay mas autos en el estacionamiento.");

			pitsCars.put("FIN");
		}

		private void goServiceBase() throws InterruptedException
		{
			String nextCar;

			nextCar = fixedCars.take();

			while(!nextCar.equals("FIN"))
			{
				serviceMTX.acquire();
				goService();
				serviceMTX.release();

				nextCar = fixedCars.take();
			}

			System.out.println("El asistente " + this.name + " avisa que no hay mas autos en las fosas.");
		}

		private void goPit() throws InterruptedException
		{
			System.out.println("El asistente " + this.name + " lleva un auto a la fosa.");

			Thread.sleep(TIME);

			pitsCars.add("AUTO");
		}

		private void goService() throws InterruptedException
		{
			System.out.println("El asistente " + this.name + " lleva un auto al servicio.");

			Thread.sleep(TIME);
		}

	}

	static class Charles extends Thread
	{
		int numEnds = 0;

		public void run()
		{
			String nextCar;

			try
			{

				while(this.numEnds < NUM_ASSISTANTS)
				{
					nextCar = pitsCars.take();

					if(nextCar.equals("FIN"))
					{
						this.numEnds ++;
					}

					else
					{
						fixCar();
					}
				}

				System.out.println("Charles avisa que no hay mas autos en las fosas.");

				for (int i = 0; i < NUM_ASSISTANTS; i ++)
				{
				    fixedCars.put("FIN");
				}
			}
			catch (InterruptedException e)
			{
				System.out.println("ERROR: Charles no pudo realizar su trabajo.");
			}
		}

		private void fixCar() throws InterruptedException
		{
			System.out.println("Charles comienza la reparación del auto.");

			Thread.sleep(TIME);

			pitsMTX.release();

			fixedCars.put("AUTO");
		}

	}

}

Writing Ejercicio3/Main.java


Finalmente, ejecutamos el programa

In [ ]:
!javac Ejercicio3/*.java

In [ ]:
!java Ejercicio3.Main

Richard lleva un auto al estacionamiento.
Richard lleva un auto al estacionamiento.
Aaron comienza la inspección del auto.
Richard lleva un auto al estacionamiento.
Aaron comienza la inspección del auto.
El asistente Bran lleva un auto a la fosa.
Richard avisa que no hay mas autos en la calle.
El asistente Rob lleva un auto a la fosa.
Aaron comienza la inspección del auto.
Charles comienza la reparación del auto.
Aaron avisa que no hay mas autos en el estacionamiento.
El asistente Bran lleva un auto a la fosa.
El asistente Rob avisa que no hay mas autos en el estacionamiento.
Charles comienza la reparación del auto.
El asistente Rob lleva un auto al servicio.
El asistente Bran avisa que no hay mas autos en el estacionamiento.
Charles comienza la reparación del auto.
El asistente Bran lleva un auto al servicio.
El asistente Bran lleva un auto al servicio.
Charles avisa que no hay mas autos en las fosas.
El asistente Rob avisa que no hay mas autos en las fosas.
El asistente Bran avisa qu